In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [2]:
team = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MTeams.csv")
seasons = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSeasons.csv")
seeds = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv")
season_results = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv")
tour_results = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv")

In [3]:
df = team.copy()
df = df.set_index("TeamID")
df["MeanWinScore"] = season_results.groupby("WTeamID").mean()["WScore"].astype(int)
df["MeanLossScore"] = season_results.groupby("LTeamID").mean()["LScore"].astype(int)
df["WinLossDiff"] = df["MeanWinScore"] - df["MeanLossScore"]


df["Wins"] = season_results["WTeamID"].value_counts() # add feature of the number of total wins
df.head(10)

,TeamName,FirstD1Season,LastD1Season,MeanWinScore,MeanLossScore,WinLossDiff,Wins
TeamID,,,,,,,
1101,Abilene Chr,2014,2022,76,63,13,109
1102,Air Force,1985,2022,70,59,11,368
1103,Akron,1985,2022,75,64,11,619
1104,Alabama,1985,2022,76,64,12,719
1105,Alabama A&M,2000,2022,74,60,14,208
1106,Alabama St,1985,2022,76,65,11,437
1107,SUNY Albany,2000,2022,71,60,11,333
1108,Alcorn St,1985,2022,78,64,14,350
1109,Alliant Intl,1985,1991,90,79,11,42


In [4]:
submission = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv")

In [5]:
years = []
teama = []
teamb = [] 
for _,row in submission.iterrows():
    years.append(int(row["ID"].split("_")[0]))
    teama.append(int(row["ID"].split("_")[1]))
    teamb.append(int(row["ID"].split("_")[2]))
submission["Season"] = years
submission["TeamA"] = teama
submission["TeamB"] = teamb

In [6]:
submission["TeamAWins"] = df.loc[submission["TeamA"].values]["Wins"].values
submission["TeamBWins"] = df.loc[submission["TeamB"].values]["Wins"].values

submission["TeamAMeanLossScore"] = df.loc[submission["TeamA"].values]["MeanLossScore"].values
submission["TeamBMeanLossScore"] = df.loc[submission["TeamB"].values]["MeanLossScore"].values


submission["TeamAMeanWinScore"] = df.loc[submission["TeamA"].values]["MeanWinScore"].values
submission["TeamBMeanWinScore"] = df.loc[submission["TeamB"].values]["MeanWinScore"].values

submission["TeamAWinLossDiff"] = df.loc[submission["TeamA"].values]["WinLossDiff"].values
submission["TeamBWinLossDiff"] = df.loc[submission["TeamB"].values]["WinLossDiff"].values


In [7]:
submission

,ID,Pred,Season,TeamA,TeamB,TeamAWins,TeamBWins,TeamAMeanLossScore,TeamBMeanLossScore,TeamAMeanWinScore,TeamBMeanWinScore,TeamAWinLossDiff,TeamBWinLossDiff
0,2016_1112_1114,0.5,2016,1112,1114,878,562,69,64,82,76,13,12
1,2016_1112_1122,0.5,2016,1112,1122,878,518,69,68,82,78,13,10
2,2016_1112_1124,0.5,2016,1112,1124,878,585,69,65,82,78,13,13
3,2016_1112_1138,0.5,2016,1112,1138,878,443,69,66,82,77,13,11
4,2016_1112_1139,0.5,2016,1112,1139,878,682,69,63,82,73,13,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2021_1452_1457,0.5,2021,1452,1457,693,506,66,63,78,75,12,12
11386,2021_1452_1458,0.5,2021,1452,1458,693,714,66,61,78,72,12,11
11387,2021_1455_1457,0.5,2021,1455,1457,662,506,61,63,75,75,14,12
11388,2021_1455_1458,0.5,2021,1455,1458,662,714,61,61,75,72,14,11
